# Astronomy Libraries Comparison

Load AA, WWT, and Skyfield JSON outputs with pandas, parse `time` as UTC datetime, attach `astropy.time.Time`, and compare values column-by-column.


In [219]:
from pathlib import Path

import numpy as np
import pandas as pd
from astropy.time import Time
from astropy.table import Table

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 220)



ROOT = Path('.').resolve()
OUTPUTS = ROOT / 'outputs'
FILES = {
    'aa': OUTPUTS / 'methods_to_check_aa_records.json',
    'wwt': OUTPUTS / 'methods_to_check_wwt_records.json',
    'skyfield': OUTPUTS / 'methods_to_check_skyfield_records.json',
}
ID_COLS = ['index', 'lat', 'lon', 'height', 'time', 'name']



In [220]:
def load(key):
    df = pd.read_json(FILES[key])
    df['time'] = df['time'].apply(lambda x: Time(x))
    # df.drop(columns=['name'], inplace=True)
    return df
df_aa = load('aa')
df_wwt = load('wwt')
df_sf = load('skyfield')

/Users/jal194/github/astronomy-libraries-test/.venv/lib/python3.13/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/jal194/github/astronomy-libraries-test/.venv/lib/python3.13/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/Users/jal194/github/astronomy-libraries-test/.venv/lib/python3.13/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


In [221]:
# --- Precision thresholds (from tests.js) ---
ANGULAR_PRECISION = 60 / 3600  # degrees (60 arcsec)
TIME_PRECISION = 60            # seconds
KM_PRECISION = 1               # km
AU_PRECISION = 100 / 1.15e8    # AU (100 km)

# --- Helper: classify columns by type ---
def classify_column(col):
    col = col.lower()

    # 360-wrap angles: RA and longitudes
    if any(x in col for x in ('rightascension', 'longitude')):
        return 'angle_circular'

    # Linear angles (no rollover)
    if any(x in col for x in ('declination', 'latitude', 'elongation', 'phaseangle')):
        return 'angle_linear'

    if 'distance' in col or 'radius' in col:
        if 'km' in col:
            return 'km'
        return 'au'

    if col.endswith('_rise') or col.endswith('_set') or col.endswith('_transit'):
        return 'time'

    return 'other'

# --- Helper: get threshold for column ---
def get_threshold(col):
    t = classify_column(col)
    if t in ('angle_circular', 'angle_linear'):
        return ANGULAR_PRECISION
    if t == 'time':
        return TIME_PRECISION / 3600
    if t == 'km':
        return KM_PRECISION
    if t == 'au':
        return AU_PRECISION
    return None

# --- Helper: compute diff with optional 360 rollover ---
def compute_diff(s1, s2, col):
    v1 = pd.to_numeric(s1, errors='coerce')
    v2 = pd.to_numeric(s2, errors='coerce')
    raw = (v1 - v2).abs()

    if classify_column(col) == 'angle_circular':
        return np.minimum(raw, 360 - raw)

    return raw

# --- Compute differences ---
def diff_and_emojis(df1, df2, label1, label2):
    result = df1[ID_COLS].copy() if all(col in df1.columns for col in ID_COLS) else pd.DataFrame(index=df1.index)
    new_cols = {}

    for col in df1.columns:
        if col in ID_COLS:
            continue
        # if 'pluto' in col:
        #     continue

        threshold = get_threshold(col)
        diff = compute_diff(df1[col], df2[col], col)

        new_cols[col] = [
            {label1: v1, label2: v2, 'log_rel_diff': float(np.around(np.log10(abs(d / v1)),2))}
            for v1, v2, d in zip(df1[col], df2[col], diff)
        ]

        if threshold is not None:
            both_nan = pd.isna(df1[col]) & pd.isna(df2[col])
            pass_col = (diff <= threshold) | both_nan
            emoji_col = pass_col.map(lambda x: '✅' if x else '❌')
            new_cols[col + '_emoji'] = emoji_col

    result = pd.concat([result, pd.DataFrame(new_cols, index=df1.index)], axis=1)
    id_cols = [col for col in ID_COLS if col in result.columns]
    other_cols = [col for col in result.columns if col not in id_cols]
    result = result[id_cols + other_cols]
    return result

def only_emoji(df):
    cols = [c for c in df.columns if 'emoji' in c]
    return df[cols]

def filter_red_x(df, drop_cols=False):
    emoji_cols = [c for c in df.columns if c.endswith('_emoji')]
    # Any row with at least one ❌ in any emoji col
    mask = df[emoji_cols].apply(lambda row: '❌' in row.values, axis=1)
    filtered = df[mask].copy()
    if drop_cols:
        # Drop non-ID columns (and their _emoji) that have no ❌ in any row
        cols_to_keep = list(ID_COLS)
        for col in df.columns:
            if col in ID_COLS or not col.endswith('_emoji'):
                continue
            if (filtered[col] == '❌').any():
                base_col = col[:-6]  # remove _emoji
                cols_to_keep.append(base_col)
                cols_to_keep.append(col)
        # Keep only ID columns and those with at least one ❌
        cols_to_keep = [c for c in cols_to_keep if c in filtered.columns]
        return filtered[cols_to_keep]
    return filtered


In [222]:
sf_aa = filter_red_x(diff_and_emojis(df_sf, df_aa, 'sf', 'aa'))
# Table.from_pandas(sf_aa).show_in_browser(jsviewer=True)
print(f"{sf_aa.to_numpy().flatten().tolist().count('❌')} errors")

1102 errors


In [223]:
sf_wwt = filter_red_x(diff_and_emojis(df_sf, df_wwt, 'sf', 'wwt'))
# Table.from_pandas(sf_wwt).show_in_browser(jsviewer=True)
print(f"{sf_wwt.to_numpy().flatten().tolist().count('❌')} errors")


1149 errors


In [224]:
aa_wwt = filter_red_x(diff_and_emojis(df_aa, df_wwt, 'aa', 'wwt'))
# Table.from_pandas(sf_wwt).show_in_browser(jsviewer=True)
print(f"{aa_wwt.to_numpy().flatten().tolist().count('❌')} errors")

748 errors


/var/folders/yl/7lc06h196yd8dcv50gljyg7h0000gq/T/ipykernel_61254/3731125484.py:68: RuntimeWarning: divide by zero encountered in log10
  {label1: v1, label2: v2, 'log_rel_diff': float(np.around(np.log10(abs(d / v1)),2))}


In [232]:
# Table.from_pandas(sf_wwt).write('sf_wwt.html', format='ascii.html', overwrite=True)
# Table.from_pandas(sf_aa).write('sf_aa.html', format='ascii.html', overwrite=True)
# Table.from_pandas(aa_wwt).write('aa_wwt.html', format='ascii.html', overwrite=True)
with open('sf_wwt.html', 'w') as f: f.write(sf_wwt.to_html())
with open('sf_aa.html', 'w') as f: f.write(sf_aa.to_html())
with open('aa_wwt.html', 'w') as f: f.write(aa_wwt.to_html())

In [226]:
# Compare which library (AA vs WWT) is closer to Skyfield for each quantity
value_cols = [c for c in df_sf.columns if c not in ID_COLS]

rows = []
for col in value_cols:
    diff_aa = compute_diff(df_sf[col], df_aa[col], col)
    diff_wwt = compute_diff(df_sf[col], df_wwt[col], col)

    aa_closer = 0
    wwt_closer = 0
    ties = 0
    for d_aa, d_wwt in zip(diff_aa, diff_wwt):
        if pd.isna(d_aa) or pd.isna(d_wwt):
            continue
        if d_aa < d_wwt:
            aa_closer += 1
        elif d_wwt < d_aa:
            wwt_closer += 1
        else:
            ties += 1

    winner = 'AA' if aa_closer > wwt_closer else ('WWT' if wwt_closer > aa_closer else 'Tie')
    rows.append({
        'column': col,
        'aa_closer': aa_closer,
        'wwt_closer': wwt_closer,
        'ties': ties,
        'mean_diff_aa': diff_aa.mean(),
        'mean_diff_wwt': diff_wwt.mean(),
        'winner': winner,
    })

winner_df = pd.DataFrame(rows)
winner_df

,column,aa_closer,wwt_closer,ties,mean_diff_aa,mean_diff_wwt,winner
0,sun_apparentGeocentricRightAscension,54,6,0,0.00162224,0.00650958,AA
1,sun_apparentGeocentricDeclination,42,18,0,0.00017242,0.00148737,AA
2,sun_rise,24,12,0,0.69861522,0.00259360,AA
3,sun_transit,26,10,0,0.60356328,0.00251324,AA
4,sun_set,10,26,0,0.62983970,0.00251420,WWT
...,...,...,...,...,...,...,...
77,pluto_transit,0,32,0,4.12548447,0.04691297,WWT
78,pluto_set,1,30,0,3.13472084,0.06454977,WWT
79,pluto_apparentGeocentricLongitude,0,60,0,22.72718376,0.62374791,WWT
80,pluto_apparentGeocentricLatitude,0,60,0,3.52141475,0.00493088,WWT


In [227]:
# Build full (unfiltered) comparison tables to get log_rel_diff for all rows
full_sf_aa = diff_and_emojis(df_sf, df_aa, 'sf', 'aa')
full_sf_wwt = diff_and_emojis(df_sf, df_wwt, 'sf', 'wwt')

# Get value columns (non-ID, non-emoji)
value_cols = [c for c in full_sf_aa.columns if c not in ID_COLS and not c.endswith('_emoji')]

# Build a wide table: rows = cases (index), columns = quantities, cells = 'AA' / 'WWT' / 'Tie'
compare_rows = []
for i in full_sf_aa.index:
    row = {c: full_sf_aa.at[i, c] for c in ID_COLS if c in full_sf_aa.columns}
    for col in value_cols:
        cell_aa = full_sf_aa.at[i, col]
        cell_wwt = full_sf_wwt.at[i, col]
        # Extract log_rel_diff from the dict
        lrd_aa = cell_aa.get('log_rel_diff', None) if isinstance(cell_aa, dict) else None
        lrd_wwt = cell_wwt.get('log_rel_diff', None) if isinstance(cell_wwt, dict) else None
        if lrd_aa is None or lrd_wwt is None or np.isnan(lrd_aa) or np.isnan(lrd_wwt):
            row[col] = '—'
        elif (lrd_aa) < (lrd_wwt):
            row[col] = 'AA'  # AA has smaller log_rel_diff (closer to SF)
        elif (lrd_wwt) < (lrd_aa):
            row[col] = 'WWT'
        else:
            row[col] = 'Tie'
    compare_rows.append(row)

compare_df = pd.DataFrame(compare_rows)
compare_df.head(5)

,index,lat,lon,height,time,name,sun_apparentGeocentricRightAscension,sun_apparentGeocentricDeclination,sun_rise,sun_transit,sun_set,sun_apparentEclipticLongitude,sun_apparentEclipticLatitude,moon_geocentricRightAscension,moon_geocentricDeclination,moon_rise,moon_transit,moon_set,moon_geocentricEclipticLongitude,moon_geocentricEclipticLatitude,moon_geocentricDistanceKm,earth_eclipticLongitude,earth_eclipticLatitude,earth_radiusVector,mercury_apparentGeocentricRightAscension,mercury_apparentGeocentricDeclination,mercury_rise,mercury_transit,mercury_set,mercury_apparentGeocentricLongitude,mercury_apparentGeocentricLatitude,mercury_apparentGeocentricDistance,venus_apparentGeocentricRightAscension,venus_apparentGeocentricDeclination,venus_rise,venus_transit,venus_set,venus_apparentGeocentricLongitude,venus_apparentGeocentricLatitude,venus_apparentGeocentricDistance,mars_apparentGeocentricRightAscension,mars_apparentGeocentricDeclination,mars_rise,mars_transit,mars_set,mars_apparentGeocentricLongitude,mars_apparentGeocentricLatitude,mars_apparentGeocentricDistance,jupiter_apparentGeocentricRightAscension,jupiter_apparentGeocentricDeclination,jupiter_rise,jupiter_transit,jupiter_set,jupiter_apparentGeocentricLongitude,jupiter_apparentGeocentricLatitude,jupiter_apparentGeocentricDistance,saturn_apparentGeocentricRightAscension,saturn_apparentGeocentricDeclination,saturn_rise,saturn_transit,saturn_set,saturn_apparentGeocentricLongitude,saturn_apparentGeocentricLatitude,saturn_apparentGeocentricDistance,uranus_apparentGeocentricRightAscension,uranus_apparentGeocentricDeclination,uranus_rise,uranus_transit,uranus_set,uranus_apparentGeocentricLongitude,uranus_apparentGeocentricLatitude,uranus_apparentGeocentricDistance,neptune_apparentGeocentricRightAscension,neptune_apparentGeocentricDeclination,neptune_rise,neptune_transit,neptune_set,neptune_apparentGeocentricLongitude,neptune_apparentGeocentricLatitude,neptune_apparentGeocentricDistance,pluto_apparentGeocentricRightAscension,pluto_apparentGeocentricDeclination,pluto_rise,pluto_transit,pluto_set,pluto_apparentGeocentricLongitude,pluto_apparentGeocentricLatitude,pluto_apparentGeocentricDistance
0,0,0.00000000,0.00000000,0,2025-03-20T09:01:00.000,equator_prime,AA,AA,WWT,WWT,WWT,Tie,AA,Tie,Tie,AA,AA,AA,WWT,Tie,Tie,Tie,WWT,Tie,WWT,WWT,AA,WWT,WWT,WWT,WWT,Tie,WWT,WWT,AA,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,AA,WWT,WWT,Tie,WWT,WWT,AA,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,Tie,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT
1,1,0.00000000,0.00000000,0,2026-01-01T12:00:00.000,equator_prime,AA,AA,AA,AA,WWT,Tie,WWT,Tie,Tie,AA,WWT,AA,WWT,Tie,Tie,Tie,WWT,Tie,WWT,AA,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,Tie,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,AA,AA,AA,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT
2,2,0.00000000,0.00000000,0,2025-06-21T02:42:00.000,equator_prime,AA,WWT,AA,AA,WWT,Tie,WWT,Tie,Tie,AA,AA,AA,WWT,Tie,Tie,Tie,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,AA,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,AA,WWT,WWT,AA,AA,AA,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,AA,WWT,AA,AA,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT
3,3,0.00000000,0.00000000,0,2025-09-22T18:19:00.000,equator_prime,AA,AA,AA,AA,WWT,Tie,WWT,Tie,Tie,AA,WWT,WWT,WWT,Tie,Tie,Tie,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,AA,WWT,WWT,AA,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT
4,4,0.00000000,0.00000000,0,2025-12-21T15:03:00.000,equator_prime,AA,WWT,AA,AA,WWT,Tie,WWT,Tie,Tie,AA,AA,AA,WWT,Tie,Tie,Tie,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,Tie,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT,WWT,WWT,WWT,AA,WWT,WWT,WWT,WWT,WWT,WWT,WWT,WWT,Tie,WWT,WWT,WWT

In [228]:
# Summary: per column, count how many cases AA vs WWT was closer (by log_rel_diff)
summary_rows = []
for col in value_cols:
    aa_closer = 0
    wwt_closer = 0
    ties = 0
    mean_lrd_aa = []
    mean_lrd_wwt = []
    for i in compare_df.index:
        val = compare_df.at[i, col]
        if val == 'AA':
            aa_closer += 1
        elif val == 'WWT':
            wwt_closer += 1
        elif val == 'Tie':
            ties += 1
        # collect log_rel_diffs for mean
        cell_aa = full_sf_aa.at[i, col]
        cell_wwt = full_sf_wwt.at[i, col]
        lrd_a = cell_aa.get('log_rel_diff', np.nan) if isinstance(cell_aa, dict) else np.nan
        lrd_w = cell_wwt.get('log_rel_diff', np.nan) if isinstance(cell_wwt, dict) else np.nan
        mean_lrd_aa.append(lrd_a)
        mean_lrd_wwt.append(lrd_w)
    winner = 'AA' if aa_closer > wwt_closer else ('WWT' if wwt_closer > aa_closer else 'Tie')
    summary_rows.append({
        'column': col,
        'aa_closer': aa_closer,
        'wwt_closer': wwt_closer,
        'ties': ties,
        'mean_log_rel_diff_aa': np.nanmean(mean_lrd_aa),
        'mean_log_rel_diff_wwt': np.nanmean(mean_lrd_wwt),
        'winner': winner,
    })

lrd_winner_df = pd.DataFrame(summary_rows)
lrd_winner_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>column</th>\n      <th>aa_closer</th>\n      <th>wwt_closer</th>\n      <th>ties</th>\n      <th>mean_log_rel_diff_aa</th>\n      <th>mean_log_rel_diff_wwt</th>\n      <th>winner</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>sun_apparentGeocentricRightAscension</td>\n      <td>54</td>\n      <td>6</td>\n      <td>0</td>\n      <td>-5.36200000</td>\n      <td>-4.54700000</td>\n      <td>AA</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>sun_apparentGeocentricDeclination</td>\n      <td>42</td>\n      <td>18</td>\n      <td>0</td>\n      <td>-4.13000000</td>\n      <td>-3.51300000</td>\n      <td>AA</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>sun_rise</td>\n      <td>24</td>\n      <td>12</td>\n      <td>0</td>\n      <td>-3.78500000</td>\n      <td>-3.89000000</td>\n      <td>AA</td>\n    </tr>\n    <tr>\n      <th>3</th>\n  

In [229]:
with open('lrd_winner.html', 'w') as f:
    f.write(lrd_winner_df.to_html())

In [230]:
lrd_winner_df['winner'].value_counts()

winner
WWT    61
Tie    11
AA     10
Name: count, dtype: int64